### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# set total number of schools and students
total_schools = len(school_data)
total_students = len(student_data)

# get total budget
total_budget = school_data['budget'].sum()

# get avg scores
avg_math_score = student_data['math_score'].mean()
avg_read_score = student_data['reading_score'].mean()

# get number of passing math students
math_pass_df = student_data.loc[(student_data['math_score'] >= 70), : ]
stu_passing_math = len(math_pass_df)

# get number of passing reading students
read_pass_df = student_data.loc[(student_data['reading_score'] >= 70), : ]
stu_passing_read = len(read_pass_df)

# get number of students passing both math and reading
overall_pass_df = pd.merge(math_pass_df, read_pass_df, how='inner', on='Student ID', suffixes=('_math', '_read'))
stu_passing_overall = len(overall_pass_df)

# calculate pass percentage by type
perc_pass_math = stu_passing_math / total_students * 100
perc_pass_read = stu_passing_read / total_students * 100
perc_pass_overall = stu_passing_overall / total_students * 100

# create data_frame
district_summary_df = pd.DataFrame([
    {
        'Total Schools': total_schools,
        'Total Students': total_students,
        'Total Budget': total_budget,
        'Average Math Score': avg_math_score,
        'Average Reading Score': avg_read_score,
        '% Passing Math': perc_pass_math,
        '% Passing Reading': perc_pass_read,
        '% Overall Passing': perc_pass_overall
    }
])

# format District Summary dataframe
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
district_summary_df['Average Math Score'] = district_summary_df['Average Math Score'].map('{:.2f}'.format)
district_summary_df['Average Reading Score'] = district_summary_df['Average Reading Score'].map('{:.2f}'.format)
district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].map('{:.2f}%'.format)
district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].map('{:.2f}%'.format)
district_summary_df['% Overall Passing'] = district_summary_df['% Overall Passing'].map('{:.2f}%'.format)

# print dataframe
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [16]:
# set base columns for School Summary DataFrame
school_summary_df = school_data[['school_name', 'type', 'size', 'budget']]
school_summary_df['avg_budget'] = school_summary_df['budget'] / school_summary_df['size']

# get avg student grades and merge back into School Summary
student_grades_df = student_data[['school_name', 'math_score', 'reading_score']]
avg_grades_df = student_grades_df.groupby('school_name').mean()
school_summary_df = school_summary_df.merge(avg_grades_df, how = 'inner', on = 'school_name')

# get passing math student information and merge back into school summary
math_pass_count_df = math_pass_df.groupby('school_name')['Student ID'].nunique()
math_pass_count_df = math_pass_count_df.rename('math_pass_count')

school_summary_df = school_summary_df.merge(math_pass_count_df, how = 'inner', on = 'school_name')

# get passing reading students information and merge back into school summary
read_pass_count_df = read_pass_df.groupby('school_name')['Student ID'].nunique()
read_pass_count_df = read_pass_count_df.rename('read_pass_count')

school_summary_df = school_summary_df.merge(read_pass_count_df, how = 'inner', on = 'school_name')

# get students passing both subjects and merge back into school summary
overall_pass_count_df = overall_pass_df.groupby('school_name_math')['Student ID'].nunique()
overall_pass_count_df = overall_pass_count_df.rename('overall_pass_count')
overall_pass_count_df = overall_pass_count_df.rename_axis('school_name')

school_summary_df = school_summary_df.merge(overall_pass_count_df, how = 'inner', on = 'school_name')

# calculate percentage of passing students
school_summary_df['perc_pass_math'] = school_summary_df['math_pass_count'] / school_summary_df['size'] * 100
school_summary_df['perc_pass_read'] = school_summary_df['read_pass_count'] / school_summary_df['size'] * 100
school_summary_df['perc_pass_overall'] = school_summary_df['overall_pass_count'] / school_summary_df['size'] * 100

# remove unnecessary columns
school_summary_df.drop(['math_pass_count', 'read_pass_count', 'overall_pass_count'], axis=1, inplace=True)

#rename column headers
school_summary_df.rename(columns={
    'school_name': 'School Name',
    'type': 'School Type',
    'size': 'Total Students',
    'budget': 'Total School Budget',
    'avg_budget': 'Per Student Budget',
    'math_score': 'Average Math Score',
    'reading_score': 'Average Reading Score',
    'perc_pass_math': '% Passing Math',
    'perc_pass_read': '% Passing Reading',
    'perc_pass_overall': '% Overall Passing'
}, inplace=True)

# set output format
output_format = {
    'Total Students': '{:,.0f}',
    'Total School Budget': '${:,.2f}',
    'Per Student Budget': '${:,.2f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2f}%',
    '% Passing Reading': '{:.2f}%',
    '% Overall Passing': '{:.2f}%'
}
school_summary_df.set_index('School Name', inplace=True)

school_summary_df.sort_index().style.format(output_format)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [21]:
top_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=False).head(5)
top_schools_df.style.format(output_format)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [20]:
top_schools_df = school_summary_df.sort_values('% Overall Passing').head(5)
top_schools_df.style.format(output_format)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# get average per school for each grade
math_9 = student_data.loc[(student_data['grade'] == '9th'), ['school_name', 'math_score']]
math_9_avg = math_9.groupby('school_name')['math_score'].mean()
math_9_avg.rename('9th', inplace=True)

math_10 = student_data.loc[(student_data['grade'] == '10th'), ['school_name', 'math_score']]
math_10_avg = math_10.groupby('school_name')['math_score'].mean()
math_10_avg.rename('10th', inplace=True)

math_11 = student_data.loc[(student_data['grade'] == '11th'), ['school_name', 'math_score']]
math_11_avg = math_11.groupby('school_name')['math_score'].mean()
math_11_avg.rename('11th', inplace=True)

math_12 = student_data.loc[(student_data['grade'] == '12th'), ['school_name', 'math_score']]
math_12_avg = math_12.groupby('school_name')['math_score'].mean()
math_12_avg.rename('12th', inplace=True)

# merge results together into dataframe
math_avg_df = math_9_avg.to_frame()
math_avg_df = math_avg_df.merge(math_10_avg, on='school_name', how='inner')
math_avg_df = math_avg_df.merge(math_11_avg, on='school_name', how='inner')
math_avg_df = math_avg_df.merge(math_12_avg, on='school_name', how='inner')

# print results
math_avg_df.style.format({
    '9th': '{:.2f}',
    '10th': '{:.2f}',
    '11th': '{:.2f}',
    '12th': '{:.2f}'
})


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# get average per school for each grade
reading_9 = student_data.loc[(student_data['grade'] == '9th'), ['school_name', 'reading_score']]
reading_9_avg = reading_9.groupby('school_name')['reading_score'].mean()
reading_9_avg.rename('9th', inplace=True)

reading_10 = student_data.loc[(student_data['grade'] == '10th'), ['school_name', 'reading_score']]
reading_10_avg = reading_10.groupby('school_name')['reading_score'].mean()
reading_10_avg.rename('10th', inplace=True)

reading_11 = student_data.loc[(student_data['grade'] == '11th'), ['school_name', 'reading_score']]
reading_11_avg = reading_11.groupby('school_name')['reading_score'].mean()
reading_11_avg.rename('11th', inplace=True)

reading_12 = student_data.loc[(student_data['grade'] == '12th'), ['school_name', 'reading_score']]
reading_12_avg = reading_12.groupby('school_name')['reading_score'].mean()
reading_12_avg.rename('12th', inplace=True)

# merge results together into dataframe
reading_avg_df = reading_9_avg.to_frame()
reading_avg_df = reading_avg_df.merge(reading_10_avg, on='school_name', how='inner')
reading_avg_df = reading_avg_df.merge(reading_11_avg, on='school_name', how='inner')
reading_avg_df = reading_avg_df.merge(reading_12_avg, on='school_name', how='inner')

# print results
reading_avg_df.style.format({
    '9th': '{:.2f}',
    '10th': '{:.2f}',
    '11th': '{:.2f}',
    '12th': '{:.2f}'
})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
spending_bins = [575, 600, 625, 650, 675]
spending_labels = ['$575-600', '$600-625', '$625-650', '$650-675']

school_summary_df['Spending Range (Per Student)'] = pd.cut(school_summary_df['Per Student Budget'], spending_bins, labels=spending_labels, include_lowest=True)
school_summary_df.groupby('Spending Range (Per Student)')['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'].mean().style.format(output_format)

c:\users\nconn\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range (Per Student),,,,,
$575-600,83.44,83.89,93.54%,96.46%,90.27%
$600-625,83.60,83.93,93.99%,96.54%,90.57%
$625-650,78.03,81.42,71.11%,83.45%,59.94%
$650-675,76.96,81.06,66.22%,81.09%,53.52%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
size_bins =[0, 1000, 2000, 5000]
size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], size_bins, labels=size_labels, include_lowest=True)
school_summary_df.groupby('School Size')['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'].mean().style.format(output_format)

c:\users\nconn\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
school_summary_df.groupby('School Type')['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'].mean().style.format(output_format)


c:\users\nconn\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
